#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [1]:
import nltk
import string
import numpy as np
import math as math
from scipy import spatial
from sklearn.cluster import KMeans
from collections import Counter
nltk.download('brown')
nltk.download('stopwords')

[nltk_data] Downloading package brown to
[nltk_data]     /home/ilangurudev/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ilangurudev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [2]:
from nltk.corpus import brown
brown_words = brown.words()

In [3]:
brown_words[1]

u'Fulton'

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [4]:
# Your Code Here...
from nltk.corpus import stopwords

stop_words = stopwords.words('english') + list(string.punctuation) + ["''", "'", '"', '""', '``', '`','--']
# filtered_brown_words = [word for word in word_list if word not in ]

filtered_brown_words = []

for word in brown_words:
    w = word.lower()
    if not w in stop_words:
        filtered_brown_words.append(w)

In [5]:
filtered_brown_words

[u'fulton',
 u'county',
 u'grand',
 u'jury',
 u'said',
 u'friday',
 u'investigation',
 u"atlanta's",
 u'recent',
 u'primary',
 u'election',
 u'produced',
 u'evidence',
 u'irregularities',
 u'took',
 u'place',
 u'jury',
 u'said',
 u'term-end',
 u'presentments',
 u'city',
 u'executive',
 u'committee',
 u'over-all',
 u'charge',
 u'election',
 u'deserves',
 u'praise',
 u'thanks',
 u'city',
 u'atlanta',
 u'manner',
 u'election',
 u'conducted',
 u'september-october',
 u'term',
 u'jury',
 u'charged',
 u'fulton',
 u'superior',
 u'court',
 u'judge',
 u'durwood',
 u'pye',
 u'investigate',
 u'reports',
 u'possible',
 u'irregularities',
 u'hard-fought',
 u'primary',
 u'mayor-nominate',
 u'ivan',
 u'allen',
 u'jr.',
 u'relative',
 u'handful',
 u'reports',
 u'received',
 u'jury',
 u'said',
 u'considering',
 u'widespread',
 u'interest',
 u'election',
 u'number',
 u'voters',
 u'size',
 u'city',
 u'jury',
 u'said',
 u'find',
 u'many',
 u"georgia's",
 u'registration',
 u'election',
 u'laws',
 u'outmoded

In [7]:
counts = Counter(filtered_brown_words)
vocab = [word_count_tuple[0] for word_count_tuple in counts.most_common(5000)]
context = [word_count_tuple[0] for word_count_tuple in counts.most_common(1000)]

In [10]:
counts.most_common(20)

[(u'one', 3292),
 (u'would', 2714),
 (u'said', 1961),
 (u'new', 1635),
 (u'could', 1601),
 (u'time', 1598),
 (u'two', 1412),
 (u'may', 1402),
 (u'first', 1361),
 (u'like', 1292),
 (u'man', 1207),
 (u'even', 1170),
 (u'made', 1125),
 (u'also', 1069),
 (u'many', 1030),
 (u'must', 1013),
 (u'af', 996),
 (u'back', 966),
 (u'years', 950),
 (u'much', 937)]

## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [11]:
co_occurence = np.empty([5000, 1000])

for vocab_word_index in range(5000):
#     print vocab_word_index
    vocab_word = vocab[vocab_word_index]
    window_words = []
    for word_index in range(len(filtered_brown_words)):
        if filtered_brown_words[word_index] == vocab_word:
            start_index = word_index - 2 
            start_index = start_index if start_index >= 0 else 0
            end_index = word_index + 2
            end_index = end_index if end_index <= len(filtered_brown_words) else len(filtered_brown_words)
            words = filtered_brown_words[start_index:end_index]
            words.remove(vocab_word)
            window_words += words
    context_words_in_window = set.intersection(set(window_words), set(context))
    window_words = [word for word in window_words if word in context_words_in_window ]
    window_count = Counter(window_words)
#     print "\n\n\n >>>>>>>>>>>>", vocab_word, window_count
    for word_window_count in window_count:
        context_index = context.index(word_window_count)
        co_occurence[vocab_word_index, context_index] = window_count[word_window_count]
#         print word_window_count, window_count[word_window_count]
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


In [12]:
co_occurence

array([[54., 63., 45., ...,  1.,  0.,  0.],
       [48., 36., 65., ...,  2.,  5.,  0.],
       [33., 52., 24., ...,  8.,  0.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [13]:
# Your Code Here...
co_occurence_prob = np.empty([5000, 1000])
for row in range(5000):
    row_sum = sum(co_occurence[row,:])
    for col in range(1000):
        co_occurence_prob[row, col] = co_occurence[row, col]/row_sum

col_sums = []
for col in range(1000):
    col_sums.append(sum(co_occurence[:, col]))

col_sums_prob = []
for col in range(1000):
    col_sums_prob.append(col_sums[col]/sum(col_sums))

## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [14]:
embedding = np.empty([5000, 1000])

for row in range(5000):
    for col in range(1000):
        val = co_occurence_prob[row, col]/col_sums_prob[col]
        embedding[row, col] = max(0, math.log(val)) if not val==0.0 else 0

## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [27]:
kmeans = KMeans(n_clusters=100, random_state=0).fit(embedding)

In [36]:
for i in range(100):
    cluster_index = kmeans.labels_==i
    if sum(cluster_index) > 25:
        print "cluster ", i
        print np.array(vocab)[cluster_index]

cluster  9
[u'world' u'life' u'great' u'american' u'part' u'upon' u'fact' u'group'
 u'social' u'present' u'order' u'rather' u'among' u'form' u'important'
 u'often' u'become' u'power' u'country' u'although' u'problem' u'sense'
 u'means' u'whole' u'law' u'human' u'action' u'history' u'experience'
 u'field' u'seems' u'free' u'political' u'question' u'making' u'whose'
 u'economic' u'individual' u'society' u'community' u'true' u'force'
 u'future' u'common' u'control' u'students' u'effect' u'art' u'value'
 u'peace' u'modern' u'situation' u'process' u'personal' u'english'
 u'nature' u'values' u'view' u'spirit' u'recent' u'particular'
 u'religious' u'terms' u'especially' u'subject' u'natural'
 u'international' u'needs' u'purpose' u'involved' u'knowledge'
 u'christian' u'certainly' u'ideas' u'due' u'moral' u'statement'
 u'questions' u'nation' u'physical' u'strength' u'merely' u'concerned'
 u'literature' u'influence' u'science' u'cause' u'direct' u'theory'
 u'clearly' u'freedom' u'movement' u'or

cluster  42
[u'brown' u'de' u'poet' u'king' u'mary' u'named' u'editor' u'judge'
 u'louis' u'morgan' u'daughter' u'jones' u'mercer' u'frank' u'career'
 u'honor' u'parker' u'fellow' u'u.' u'occasion' u'lewis' u'practically'
 u'arthur' u'warren' u'ring' u'fears' u'angeles' u'representative'
 u'italian' u'wearing' u'wright' u'watson' u'lucy' u'brain' u'alexander'
 u'edward' u'admitted' u'driven' u'lawyer' u'adams' u'hardy' u'engineer'
 u'carl' u'orleans' u'dean' u'grace' u'senator' u'walter' u'pope' u'self'
 u'robinson' u'sister' u'mayor' u'reception' u'paul' u'thompson' u'chose'
 u'holmes' u'roberts' u'colonel' u'accompanied' u'columns' u'maris'
 u'baker' u'peter' u'jackson' u'occupied' u'visiting' u'clark' u'atlanta'
 u'harry' u'stanley' u'tells' u'lee' u'johnson' u'samuel' u'senior'
 u'advised' u'funeral' u'remarked' u'chandler' u'van' u'williams'
 u'harold' u'howard' u'dancer' u'dancers' u'sen.' u'historian' u'o.'
 u'albert' u'mickey' u'ear' u'hal' u'harris' u'appointment' u'freight'
 

[u'school' u'members' u'college' u'board' u'department' u'university'
 u'secretary' u'committee' u'county' u'club' u'member' u'association'
 u'student' u'democratic' u'council' u'chicago' u'announced' u'affairs'
 u'catholic' u'official' u'faculty' u'texas' u'declared' u'officials'
 u'library' u'organized' u'republican' u'welfare' u'automobile'
 u'institute' u'pennsylvania' u'representatives' u'campus' u'elected'
 u'panel' u'carleton' u"women's" u'voted' u'trustees' u'advisory'
 u'publications' u'receives']
cluster  82
[u'mr.' u'mrs.' u'john' u'dr.' u'william' u'george' u'j.' u's.' u'a.'
 u'director' u'james' u'thomas' u'charles' u'c.' u'e.' u'w.' u'robert'
 u'henry' u'b.' u'jr.' u'h.' u'richard' u'chairman' u'd.' u'm.' u'r.'
 u'f.' u'p.' u'l.' u'joseph' u'smith' u'g.' u'lawrence' u'n.' u't.']
cluster  84
[u'district' u'army' u'leaders' u'manager' u'post' u'traditional'
 u'village' u'headquarters' u'highest' u'senate' u'entitled' u'martin'
 u'executive' u'positions' u'papers' u'abroad' 

I have displayed the clusters that have more than 25 words in them. The first cluster has words like world, life, economic, political, community, society, religion etc. This makes sense as they are words that are used together often and are spatially similar. Similarly the second cluster is very scientific. It has words like machine, experiment, formula, electric, etc. 

In [25]:
# Your Code Here...
1 - spatial.distance.cosine([1,2,3,0,0,0], [1,2,3,0,0,0])

most_common_words = [i[0] for i in counts.most_common(20)]

for word in most_common_words:
    word_index = vocab.index(word)
    word_distances = []
    for row in range(5000):
        distance = spatial.distance.cosine(embedding[row,:], embedding[word_index, :])
        word_distances.append(distance)
    closest = [x for y,x in sorted(zip(word_distances, vocab))]
    print word, closest[1:10]

one [u'every', u'another', u'next', u'last', u'took', u'man', u'see', u'think', u'first']
would [u'could', u'might', u'want', u'must', u'anything', u'wanted', u'going', u"can't", u'much']
said [u'tell', u'know', u'told', u'think', u'asked', u'got', u'say', u'sure', u'see']
new [u'city', u'world', u'york', u'since', u'state', u'business', u'development', u'early', u'central']
could [u'would', u'might', u'want', u"can't", u'ever', u'wanted', u'knew', u'us', u'going']
time [u'long', u'day', u'since', u'week', u'year', u'first', u'hours', u'way', u'life']
two [u'three', u'several', u'four', u'many', u'six', u'five', u'seven', u'ago', u'different']
may [u'might', u'also', u'would', u'must', u'shall', u'possible', u'however', u'whether', u'large']
first [u'second', u'next', u'last', u'every', u'final', u'long', u'third', u'later', u'home']
like [u'said', u'know', u'knew', u'saw', u'come', u'around', u'good', u'little', u'thought']
man [u'woman', u'boy', u'never', u'old', u'see', u'wife', u'g

The frequent words' closest words make intuitive sense. 'One' is close to words like every, another, next and 'would' is close to could, might, must. 

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

# Note to the TA and Prof:

### I initially tried doing the whole Collaborative Filtering part of the homework in Python but I ran into a lot of problems that I was unable to troubleshoot (the code also took too long to run and that hindered the troubleshooting). I have attached the python code in the annex that I tried as proof of my attempt. 
### Being very comfortable in R, I decided to try it out in R and I implemented it. I am attaching the R code as both a script and snippets in the notebook and the output at every step (I'll display the tables and matrices in python). If there are any clarifications/questions in the validity of my procedure, I can always clarify. My email is ilan50_guru@tamu.edu. I hope that is not a problem. 

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

## R code to create dataframes that have sequential user and item IDs.

```
library(tidyverse)

df_train <- read_delim("TrainingRatings.txt", delim = ", ", col_names = c("item", "user", "rating"))
df_test <- read_delim("TestingRatings.txt", delim = ", ", col_names = c("item", "user", "rating"))

num_ts <- 5000

id_lookup <- 
  tibble(user =  unique(df_test$user)) %>% 
  mutate(user_id = row_number()) %>% 
  filter(user_id <= num_ts)

item_id_lookup <- 
  tibble(item =  unique(df_train$item)) %>% 
  mutate(item_id = row_number()) 

df_test_fil <- 
  df_test %>% 
  inner_join(id_lookup) %>% 
  left_join(item_id_lookup)

df_train_fil <- 
  df_train %>% 
  left_join(id_lookup)

df_train_fil_na <- 
df_train_fil %>% 
  filter(is.na(user_id)) 

id_lookup_tr <- 
  tibble(user = unique(df_train_fil_na$user)) %>% 
  mutate(user_id = row_number(),
         user_id = user_id + num_ts)

id_lookup_tr <- id_lookup_tr %>% bind_rows(id_lookup)

df_train_fil <- 
  df_train %>% 
  left_join(id_lookup_tr) %>% 
  left_join(item_id_lookup) %>% 
  mutate(no = row_number())

write_csv(df_train_fil, "df_train_fil.csv")
```

### Output:

In [9]:
import pandas as pd
import numpy as np
pd_df_train = pd.read_csv("netflixdataset/netflix-dataset/df_train_fil.csv")
print pd_df_train.head()
print pd_df_train.shape

   item     user  rating  user_id  item_id  no
0     8  1744889       1   3671.0        1   1
1     8  1395430       2   5001.0        1   2
2     8  1205593       4   5002.0        1   3
3     8  1488844       4   5003.0        1   4
4     8  1447354       1   5004.0        1   5
(3255352, 6)


## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. 
It may take long to run but you won't have memory issues. 

Set k to 0.1. 

## Creating the User Item Matrix
```
num_users_tr = length(unique(df_train_fil$user_id))
num_items = length(unique(df_train_fil$item_id))

user_item_matrix_tr = matrix(nrow = num_users_tr, ncol = num_items)

df_fill_mat <- 
df_train_fil %>% 
  select(no, item_id, user_id, rating)

for(i in 1:nrow(df_fill_mat)){
   x <- df_fill_mat[i, ]
   user_item_matrix_tr[x$user_id, x$item_id] <- x$rating
   # print(i)
}

MASS::write.matrix(user_item_matrix_tr, "user_item.csv", sep = ",")

```

### Output:

In [8]:
pd_user_item_matrix = pd.read_csv("netflixdataset/netflix-dataset/user_item.csv", header = None)
print pd_user_item_matrix.head()
print pd_user_item_matrix.shape

   0     1     2     3     4     5     6     7     8     9     ...   1811  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   3.0  ...    NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
3   NaN   4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...    NaN   

   1812  1813  1814  1815  1816  1817  1818  1819  1820  
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   2.0   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 1821 columns]
(28978, 1821)


## Creating the user similarity matrix
```
similarity <- matrix(nrow = num_ts, ncol = num_users_tr)

for(i in 1:num_ts){
  print(i)
  for(j in 1:num_users_tr){
    if(i == j){
      similarity[i,j] <- 1
    }  else{
      similarity[i,j] <- cor(user_item_matrix_tr[i,],
                             user_item_matrix_tr[j,],
                             use = "pairwise.complete.obs",
                             method = "pearson")
    }
  }
}

MASS::write.matrix(similarity, "similarity.csv", sep = ",")

```

### Output:

In [3]:
#reading in just 5 rows/5000 as the matrix is huge
pd_similarity = pd.read_csv("netflixdataset/netflix-dataset/similarity.csv", header = None, nrows = 5)
print pd_similarity.head()

      0         1         2         3         4         5         6      \
0  1.000000  0.063332  0.152727  0.369466  0.103333  0.495942 -0.047369   
1  0.063332  1.000000 -0.153141  0.171728 -0.357820 -0.049086  0.478825   
2  0.152727 -0.153141  1.000000  0.313819  0.123051  0.422608  0.188738   
3  0.369466  0.171728  0.313819  1.000000 -0.061285  0.258705  0.166528   
4  0.103333 -0.357820  0.123051 -0.061285  1.000000  0.063074 -0.069432   

      7         8         9        ...        28968     28969     28970  \
0  0.274097  0.068575 -0.073512    ...     0.093077  0.004719 -0.074619   
1  0.070912  0.240663  0.590523    ...     0.370225 -0.153659 -0.107733   
2 -0.033389  0.041667  0.269152    ...     0.266695 -0.183472  0.256490   
3  0.235566  0.131902  0.081862    ...     0.163300  0.082376  0.254706   
4  0.497007 -0.409461 -0.028807    ...     0.085280  0.169558  0.226771   

      28971     28972     28973     28974     28975     28976     28977  
0  0.085746  0.345487  0

## Predicting the ratings using the training set

```
mean_ratings = rowMeans(user_item_matrix_tr, na.rm = T)

calc_rating <- function(user_id, item_id){
  # browser()
  active_user_mean <- mean_ratings[user_id]
  active_user_similarity <- similarity[user_id, ]
  item_ratings <- user_item_matrix_tr[,item_id]
  w_r <- sum(active_user_similarity * (item_ratings - mean_ratings), na.rm = T)
  active_user_mean + (w_r/sum(abs(active_user_similarity), na.rm = T))
}

(df_test_fil <- 
  df_test_fil %>% 
  mutate(pred = map2_dbl(user_id, item_id, calc_rating)))
  
 write_csv(df_test_fil, "df_test_fil_1.csv")
```

## Output:

In [4]:
pd_df_test_fil_1 = pd.read_csv("netflixdataset/netflix-dataset/df_test_fil_1.csv")
print pd_df_test_fil_1.head()
print pd_df_test_fil_1.shape

   item     user  rating  user_id  item_id      pred
0     8   573364       1        1        1  3.452032
1     8  2149668       3        2        1  3.412368
2     8  1089184       3        3        1  2.925856
3     8  2465894       3        4        1  3.202839
4     8   534508       1        5        1  3.083743
(24814, 6)


## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

```
(df_test_fil <- 
  df_test_fil %>% 
  mutate(abs_diff = abs(rating - pred),
         squared_error = abs_diff^2))
         
 write_csv(df_test_fil, "df_test_fil_2.csv")
```

In [7]:
pd_df_test_fil_2 = pd.read_csv("netflixdataset/netflix-dataset/df_test_fil_2.csv")
print pd_df_test_fil_2.head()

   item     user  rating  user_id  item_id      pred  abs_diff  squared_error
0     8   573364       1        1        1  3.452032  2.452032       6.012462
1     8  2149668       3        2        1  3.412368  0.412368       0.170047
2     8  1089184       3        3        1  2.925856  0.074144       0.005497
3     8  2465894       3        4        1  3.202839  0.202839       0.041144
4     8   534508       1        5        1  3.083743  2.083743       4.341985


## Mean Absolute Error

In [10]:
np.mean(pd_df_test_fil_2.abs_diff)

0.7507447386771522

## Root Mean Squared Error

In [13]:
np.sqrt(np.mean(pd_df_test_fil_2.squared_error))

0.9493261563321215

## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

### The possible models that could be tried are:

1. Item-Item CF
2. Latent Factor models
3. Learned Weights instead of simialrity

# Appendix

The python code I tried initially:

In [69]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
# Your Code Here...
import pandas as pd
import math
import scipy.stats as stats
import numpy as np

df_train = pd.read_csv("netflixdataset/netflix-dataset/TrainingRatings.txt", header = None, names = ['movie', 'user', 'rating'])
df_test = pd.read_csv("netflixdataset/netflix-dataset/TestingRatings.txt", header = None, names = ['movie', 'user', 'rating'])

In [67]:
print (df_test.shape)
print (df_train.shape)

(100478, 3)
(3255352, 3)


In [70]:
users_unique_ts = df_test.user.unique()
items_unique_ts = df_test.movie.unique()

n_users_ts = users_unique_ts.shape[0]
n_items_ts = items_unique_ts.shape[0]

alternate_user_ids_ts = np.array(range(n_users_ts))
alternate_item_ids_ts = np.array(range(n_items_ts))

userid_alternate_lookup_ts = pd.DataFrame({'user_id': alternate_user_ids_ts, "user": users_unique_ts})
itemid_alternate_lookup_ts = pd.DataFrame({'item_id': alternate_item_ids_ts, "movie": items_unique_ts})

df_test = pd.merge(df_test, userid_alternate_lookup_ts, on='user')
df_test = pd.merge(df_test, itemid_alternate_lookup_ts, on='movie')

n_users_ts = 5000
n_items = 1821

df_test = df_test[df_test.user_id < n_users_ts]

df_train = pd.merge(df_train, userid_alternate_lookup_ts, on='user')
df_train = pd.merge(df_train, itemid_alternate_lookup_ts, on='movie')

df_train = df_train[df_train.user_id < n_users_ts]

In [73]:
# create nones
user_item_matrix_tr = [[None] * n_items] * n_users_ts
user_item_matrix_tr = np.array(user_item_matrix_tr)
# user_item_matrix_tr = np.zeros((n_users_tr, n_items_tr))
for row in df_train.itertuples():
    user_item_matrix_tr[row[4], row[5]] = row[3]


In [104]:
user_item_matrix_ts = [[None] * n_items] * n_users_ts
user_item_matrix_ts = np.array(user_item_matrix_ts)
# user_item_matrix_ts = np.zeros((n_users_ts, n_items))
for row in df_test.itertuples():
    user_item_matrix_ts[row[4], row[5]] = row[3]
    
A = pd.DataFrame(user_item_matrix_tr)
A = A.T
A = A.apply(pd.to_numeric, errors='ignore')
similarity = A.corr()
similarity = similarity.as_matrix()

ratings_mean_tr = A.mean()

In [79]:
pred_ratings_tr = np.zeros((n_users_ts, n_items))
ratings_mean_tr = np.array(ratings_mean_tr)
# similarity = similarity.as_matrix()
for i in range(n_items_ts):
    mean_a = ratings_mean_tr[i]
    active_user_similarity = np.array(similarity[i, :])
    s = np.nansum(np.absolute(active_user_similarity))
    if s == 0.0:
        s = 0.005 # active_user_similarity = active_user_similarity/s
    for j in range(n_items):
        if not user_item_matrix_tr[i, j] is None:
            continue
        else:
            item_ratings = np.array(user_item_matrix_tr[:, j], dtype='float')
            scores = np.array(item_ratings - ratings_mean_tr) * active_user_similarity
            pred_ratings_tr[i, j] = mean_a +(1/s)*np.nansum(scores)
            if i%100 == 0 and j%500 == 0:
                print i, j, pred_ratings_tr[i, j]

0 0 3.4507899468586616
0 500 3.4004121505783447
0 1000 3.5104999439264324
0 1500 3.4869449721546295
100 0 3.282977644394311
100 500 3.2267991504047635
100 1000 3.330431795741601
100 1500 3.2866004839850147
200 500 3.662254162212837
200 1000 3.7849452554217033
200 1500 3.7303370392248265
300 0 3.469022949469432
300 1000 3.5456647675750066
300 1500 3.513340513475011
400 0 4.267813392068516
400 500 4.260445049555403
400 1000 4.274815859781199
400 1500 4.2454386079815585
500 0 4.120283650023846
500 1500 4.154146329524107
600 0 3.5842562198400056
600 500 3.515392474772452
600 1000 3.658965080867437
600 1500 3.6168407627076
700 0 3.855313174713073
700 500 3.793267972854932
700 1500 3.885099534966553
800 0 3.295886070215344
800 500 3.2350470365358395
800 1000 3.3622448004544974
800 1500 3.324016246579882
900 0 3.8412338082051276
900 500 3.802505018101413
900 1000 3.883375056884165
900 1500 3.87118766891635
1000 0 3.230656190827719
1000 500 3.2055805859266178
1000 1000 3.224268324838564
1000 1

In [90]:
absolute_error = []
for i in range(n_users_ts):
    for j in range(n_items):
        if not user_item_matrix_ts[i,j] == 0:
             absolute_error.append(abs(user_item_matrix_ts[i,j] - pred_ratings_tr[i,j]))

In [92]:
# np.square(pred_ratings_tr - user_item_matrix_ts)
absolute_error = np.array(absolute_error, dtype='float')
np.mean(absolute_error)
np.sqrt(np.mean(np.square(absolute_error)))